<a href="https://colab.research.google.com/github/techmoksha/iisc-cds-modules/blob/main/M1_AST_03_Statistical_Testing_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Assignment 03: Statistical Testing

## Learning Objectives

At the end of the experiment, you will be able to

* understand the different viewpoints of the frequentist approaches
* understand the basic idea behind statistical hypothesis testing
* perform one-sample z-test for mean
* perform two-sample z-test for mean
* perform paired z-test
* perform two-sample t-test for mean

In [ ]:
#@title Walkthrough Video
from IPython.display import HTML
HTML("""<video width="420" height="240" controls>
<source src="https://cdn.exec.talentsprint.com/content/M1_AST6_Statistical_Testing.mp4">
</video>""")

## Information

**Frequentist inference** is a collection of error probabilistic methods which allows us to learn from data about the true state of nature in the presence of uncertainty by using model-based inference. Its core goal involves providing error control in the face of uncertainty. It was developed in the early 20-th century by Fisher, Neyman & Pearson, and others, largely replacing the present approaches.

**Confidence Interval** estimate a parameter by specifying a range of possible values. Such an interval is associated with a confidence level, which is the probability that the procedure used to generate the interval will produce an interval containing the true parameter.

To know more about confidence interval, click [here](https://www.statology.org/confidence-intervals-python/).

### Setup Steps:

In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2417594" #@param {type:"string"}

In [2]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "7219510056" #@param {type:"string"}

In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "M1_AST_03_Statistical_Testing_A" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/CDS/Datasets/2019_Data_Professional_Salary_Survey_Responses.xlsx")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://learn-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


#### Importing required packages

In [4]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm                             # A normal continuous random variable
from numpy.random import seed                            # set the randomness
from scipy import stats                                  # statistical operations
import warnings
warnings.simplefilter("ignore")

# Part-A

## Hypothesis Testing

**Hypothesis testing** is a statistical method for testing a claim or hypothesis  about a parameter in a population, using data measured in a sample. It is basically an assumption that we make about the population parameter.
Based on this, we define the

* null hypothesis: $H_0$
  
  - the null hypothesis is a general statement or default position that there is no relationship between two measured phenomena, i.e. no association among groups
* alternative hypothesis: $H_A$ or $H_1$

  - The alternative hypothesis is contrary to the null hypothesis. It is usually taken to be that the observations are the result of a real effect (with some amount of chance variation superposed)

**Significance level** (alpha ($\alpha$)) is the probability that you will make the mistake of rejecting the null hypothesis when in fact it is true.

$$\alpha = P(Rejecting\ a\ null\ hypothesis\ |\ null\ hypothesis\ is\ true)$$
<br>

The **p-value** measures the probability of getting a more extreme value than the one you got from the experiment.

$$p - value = P(Observing\ test\ statistics\ value\ |\ null\ hypothesis\ is\ true)$$
<br>

|Criteria| | Decision|
|--------|----|---------|
|p-value $\le$ α | | Reject the null hypothesis|
|p-value $\gt$ α| | Retain (or fail to reject) the null hypothesis|  

<br>

**Z-test and t-test** are two widely known hypothesis testing types.

<br>

|Basis for Comparison |  |	Z-Test |  |	t-Test |
|--------|----|----|---|---------|
|Meaning |  |	Z-test implies a hypothesis test which  |  | T-test refers to a type of parametric test that is|
|       |  |ascertains if the means of two datasets are different  |  | applied to identify, how the means of two sets of data |
|       |  | from each other when variance is given. |  |	  differ from one another when variance is not given.|
| Based on |  |	Normal distribution |  |	Student-t distribution |
| Population variance |  |	Known |  |	Unknown |
| Sample Size |  |	Large |  |	Small |
  .

### One-Sample Z-Test for Mean

The One-Sample Z-test is used to know if the difference between the sample mean and the mean of a population is large enough to be statistically significant (i.e. unlikely to have occurred by mere chance).

Assumptions
1.       Mean and variance of the population are known.

2.       The test statistic follows normal distribution.

The test statistics for Z-test is given by

$$Z- statistic = \frac{\bar{X} - \mu}{\sigma / \sqrt{n}}$$

where
- $\bar{X}$: sample mean
- $\mu$: population mean
- $\sigma$: population standard deviation
- $n$: sample size



**Example 1:** Let's create some dummy age data for the population of voters in the entire country and a sample of voters in `North_Carolina` and test whether the average age of voters in `North_Carolina` differs from the entire country population.

In [6]:
np.random.seed(6)

# Generate Population ages data, see this https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html
# .rvs provides random samples; scale = std. deviation, loc = mean, size = no. of samples

population_ages1 = stats.norm.rvs(scale=18, loc=45, size=150000)
population_ages2 = stats.norm.rvs(scale=18, loc=20, size=100000)
population_ages = np.concatenate((population_ages1, population_ages2))

# Generate North Carolina sample ages data

North_Carolina_ages1 = stats.norm.rvs(scale=18, loc=40, size=30)
# YOUR CODE HERE to create 'North_Carolina_ages2' using std=18, mean=20, sample size=25)
North_Carolina_ages2 = stats.norm.rvs(scale=18, loc=20, size=25)
North_Carolina_ages = np.concatenate((North_Carolina_ages1, North_Carolina_ages2))

print("Mean age of population of voters in the entire country: ", population_ages.mean())
print("Mean age of North Carolina population: ", North_Carolina_ages.mean())

Mean age of population of voters in the entire country:  34.97043368335889
Mean age of North Carolina population:  28.52661116077034


Notice that we used a slightly different combination of distributions to generate the sample data for `North_Carolina`, so we know that the two means are different. Let's conduct a z-test at a 95% confidence level and see if it correctly rejects the null hypothesis that the sample comes from the same distribution as the population. To conduct a one-sample z-test, we can use the `stats.ztest()` function by passing argument `alternative = 'smaller'`.

Some of the parameters of `stats.ztest()` function are:
- **x1**: first of the two independent samples
- **x2**: second of the two independent samples
- **value**:
    - In the one-sample case, value is the mean of x1 under the Null hypothesis.
    - In the two-sample case, value is the difference between mean of x1 and mean of x2 under the Null hypothesis.
- **alternative**: The alternative hypothesis, H1, has to be one of the following
    - 'two-sided': H1: difference in means not equal to value (default)
    - 'larger': H1: difference in means larger than value
    - 'smaller': H1: difference in means smaller than value

In [34]:
# Perform Z-test for mean
one_sample_ztest = sm.stats.ztest(x1= North_Carolina_ages, x2=None, value= population_ages.mean(), alternative='two-sided')

print("test-statistic (z-score): ", one_sample_ztest[0])
print("p-value: ", one_sample_ztest[1])

test-statistic (z-score):  -2.2814904209459996
p-value:  0.022519442902199885


In [35]:
# Since p < alpha, reject null hypothesis

The z-score tells us how many standard deviations from the mean our score is. In this example, our score is -2.28 [standard deviations below the mean](https://cdn.iisc.talentsprint.com/CDS/Images/ZScores.jpg). Also,  we can reject the null hypothesis that the mean of samples is equal to the population mean (i.e. 28.52) as we can see that the p-value is less than 0.05 (i.e. 0.011). This indicates that the difference between the sample mean and population mean is statistically significant.

# Part-B

### Two-Sample Z-Test for Mean

The Two-Sample Z-test is used to compare the means of two samples to see if it is feasible that they come from the same population. The null hypothesis is: the population means are equal. The **Z-test is preferred to the t-test for large samples (N > 30)  or when the variance is known**, otherwise, the sample standard deviation is a more biased estimate of a population standard deviance than is allowable, and using a two-sample t-test should be considered.

Assume that $\mu_1$ and $\mu_2$ are the population means. Our interest is to check a hypothesis on difference between $\mu_1$ and $\mu_2$, that is ($\mu_1$ - $\mu_2$). If $\bar{X_1}$ and $\bar{X_2}$ are the estimated mean values from two samples, the statistic ($\bar{X_1} - \bar{X_2}$) follows a standard normal distribution with mean ($\mu_1$ - $\mu_2$) and standard deviation
$\sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}$, where $n_1$
 and $n_2$ are the sample sizes of two samples.

The corresponding Z-statistic is given by

$$Z = \frac{(\bar{X_1} - \bar{X_2}) - (\mu_1 - \mu_2)}{\sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}}$$

**Example 2:** Let's create a sample of voters in `South_Carolina` and test whether the average age of voters in `North_Carolina` differs from the average age of voters in `South_Carolina`.

In [9]:
np.random.seed(12)

# Generate South Carolina sample ages data
South_Carolina_ages1 = stats.norm.rvs(scale=15, loc=33, size=30)
# YOUR CODE HERE to create 'South_Carolina_ages2' with std=15, mean=20, sample size=25
South_Carolina_ages2 = stats.norm.rvs(scale=15, loc=20, size=25)
# YOUR CODE HERE to create 'South_Carolina_ages' by concatenating 'South_Carolina_ages1' and 'South_Carolina_ages2'
South_Carolina_ages = np.concatenate((South_Carolina_ages1, South_Carolina_ages2))

print("Mean age of South Carolina population: ", South_Carolina_ages.mean())

Mean age of South Carolina population:  24.970648445336874


In [11]:
# Perform Two sample Z-test for Mean
two_sample_ztest = sm.stats.ztest(x1 = North_Carolina_ages,
                                  x2 = South_Carolina_ages,
                                  value = 0,
                                  alternative = 'two-sided')    # Assume samples have equal variance
print("test-statistic (z-score): ", two_sample_ztest[0])
# YOUR CODE HERE to show p-value
print("p-value: ", round(two_sample_ztest[1], 4))

test-statistic (z-score):  0.9604812179337472
p-value:  0.3368


The test yields a p-value of 0.336, which means there is a 33.6% chance we'd see the difference in the means of the two sample data are not equal to value (here, it is 0 as here we calculate the difference of the two sample means), if these two groups tested are actually identical. If we were using a 95% confidence level we would fail to reject the null hypothesis, since the p-value is greater than the corresponding significance level of 5%.

### Paired Z-Test for Mean

The paired z-test is used to test whether the mean difference of two populations is greater than, less than, or not equal to 0.

The objective is to check whether the difference in the parameter values is statistically significant before and after the intervention or between two different types of interventions.

**Example 3:** Consider the weights of a group of patients before and after an exercise program.

  - Observation 1: The weight of a group of patients was evaluated at baseline.
  - Observation 2: This same group of patients was evaluated after an 8-week exercise program.
  - Variable of interest: Body weight.

In this example, we have one group with two observations, meaning that the data are paired. In this example, we know the population variance of body weights of each patient from previous studies.

The null hypothesis is that there will be no difference in weights measured before and after the exercise program. After observing that our data meet the assumptions of a paired samples z-test (the variable of interest is continuous, it has only two groups i.e. two measurements from a single group, it has  paired samples, it has a normal variable of interest and the population variance is known), we proceed with the analysis.

When we run the analysis, we get a test statistic (in this case a Z-statistic) and a p-value.

  - The test statistic is a measure of how different the group is on our body weight variable of interest across the two observations.
  - The p-value is the chance of seeing our results assuming the exercise program actually doesn’t do anything. A p-value less than or equal to 0.05 means that our result is statistically significant and we can trust that the difference is not due to chance alone.



In [12]:
np.random.seed(11)

# Generate Weights of population before exercise program
before= stats.norm.rvs(scale=10, loc=50, size=100)

# Generate Weights of population after exercise program
after = before + stats.norm.rvs(scale=5, loc=-3.5, size=100)

# Create dataframe
weight_df = pd.DataFrame({"weight_before":before,
                          "weight_after":after,
                          "weight_change":after-before})

# YOUR CODE HERE to show few rows of 'weight_df'
weight_df.sample(10)

,weight_before,weight_after,weight_change
49,38.142447,37.167557,-0.974890
24,56.300798,47.156326,-9.144472
51,56.955673,50.080571,-6.875101
89,48.694585,39.618399,-9.076186
9,39.343970,35.658636,-3.685335
15,38.805247,38.200103,-0.605144
63,49.282506,42.287647,-6.994858
2,45.154349,33.459091,-11.695258
56,62.091016,61.141157,-0.949860
91,47.487231,43.751068,-3.736163


In [13]:
# Check summary of the data
weight_df.describe()

,weight_before,weight_after,weight_change
count,100.000000,100.000000,100.000000
mean,50.115182,46.634807,-3.480375
std,9.377513,10.423714,4.783696
min,23.466814,16.730301,-13.745286
25%,43.473681,39.362388,-6.296211
50%,50.276935,47.179677,-3.663463
75%,56.879048,53.765628,-0.511327
max,71.566744,71.336869,7.509282


The summary shows that patients lost about 3.48 pounds on average after the exercise program. Let's conduct a paired z-test to see whether this difference is significant at a 95% confidence level

In [14]:
# Perform paired Z-test
pair_ztest = sm.stats.ztest(x1 = before, x2 = after, value = 0, alternative='two-sided')
# YOUR CODE HERE to show test-statistic value for pair ztest
print("test-statistic (z-score): ", pair_ztest[0])
# YOUR CODE HERE to show p-value for pair ztest
print("p-value: ", round(pair_ztest[1], 4))

test-statistic (z-score):  2.48223901119329
p-value:  0.0131


Here the null-hypothesis was that in the population (from which the samples are drawn) the difference between similarly paired observations is 0 (as indicated by `value = 0`). The p-value in the test output shows that the probability that the difference in means is not equal to the value (0) is 1.3%, i.e. < 5%, so the null hypothesis is rejected. The differences in means of the weight measurements before and after the exercise program is statistically significant.

### Two Sample T-test for Mean

Let's see the hypothesis test for difference in two population means when the **standard deviations of the populations are unknown**. Hence we need to estimate them from the samples drawn from these two populations. An additional assumption we make here is that the standard deviation of two populations are equal (however, unknown). Then the sampling distribution of the difference in estimated means $(\bar{X_1} - \bar{X_2})$ follows a t-distribution with ($n_1
 + n_2  – 2$) degrees of freedom with mean ($\mu_1 – \mu_2$) and standard deviation
$\sqrt{S_P^2 \big( \frac{1}{n_1} + \frac{1}{n_2} \big)}$

where $S_P^2$ is the pooled variance of two samples and is given by
$S_P^2 = \frac{(n_1 - 1)S_1^2 + (n_2 - 1)S_2^2}{(n_1 + n_2 - 2)}$

The corresponding *t-statistic* is
$$t = \frac{(\bar{X_1} - \bar{X_2}) - (\mu_1 - \mu_2)}{\sqrt{S_P^2 \big( \frac{1}{n_1} + \frac{1}{n_2} \big)}}$$

where
- $\bar{X_1}$: sample mean for first population
- $\bar{X_2}$: sample mean for second population
- $n_1$: sample size for first population
- $n_2$: sample size for second population

**Example 4:** Consider the Professional Salary Survey Results dataset.
At $\alpha$ = 0.05, test whether the salary means for both male and female employees for the year 2019 who belong to the United States, are equal.

The corresponding null hypothesis and alternate hypothesis are as follows:

- $H_0$: both salary means( male and female) are equal
- $H_A$: both means are not equal

Now, we will carry out the two-sample t-test with the `stats.ttest_ind()` function.

#### Dataset Description

Dataset chosen here is [Professional Salary Survey Results dataset](https://data.world/search?preview=finance%2Fdata-professional-salary-survey&q=Professional+Salary+Survey+Results+dataset.&scope=_all). The dataset is made up of 6893 records and 29 columns. It includes important attributes of the employments. Some of the features are

- Survey Year
- SalaryUSD: salary in US dollars
- Country
- Education
- EmploymentSector
- LookingForAnotherJob
- Gender

Here we will only be considering the United States employees' Salary for the year 2019.

In [15]:
# Read data
raw_data = pd.read_excel('2019_Data_Professional_Salary_Survey_Responses.xlsx', header = 3)
raw_data.head()

,Survey Year,Timestamp,SalaryUSD,Country,PostalCode,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,...,HoursWorkedPerWeek,TelecommuteDaysPerWeek,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed,Counter
0,2017,2017-01-05 05:10:20.451,200000,United States,Not Asked,Microsoft SQL Server,10,MySQL/MariaDB,Full time employee,DBA,...,45,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
1,2017,2017-01-05 05:26:23.388,61515,United Kingdom,Not Asked,Microsoft SQL Server,15,"Oracle, PostgreSQL",Full time employee,DBA,...,35,2,Not Asked,Private business,No,Not Asked,Not Asked,Not Asked,Not Asked,1
2,2017,2017-01-05 05:32:57.367,95000,Germany,Not Asked,Microsoft SQL Server,5,"Oracle, MySQL/MariaDB, Informix",Full time employee,Other,...,45,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
3,2017,2017-01-05 05:33:03.316,56000,United Kingdom,Not Asked,Microsoft SQL Server,6,NaN,Full time employee,DBA,...,40,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
4,2017,2017-01-05 05:34:33.866,35000,France,Not Asked,Microsoft SQL Server,10,Oracle,Full time employee of a consulting/contracting...,DBA,...,40,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1


In [16]:
# Shape of data
# YOUR CODE HERE
raw_data.shape

(6893, 29)

In [17]:
# Check for missing values
raw_data.isna().sum() / len(raw_data)

,0
Survey Year,0.000000
Timestamp,0.000000
SalaryUSD,0.000000
Country,0.000000
PostalCode,0.139127
PrimaryDatabase,0.000000
YearsWithThisDatabase,0.000000
OtherDatabases,0.199188
EmploymentStatus,0.000000
JobTitle,0.000000


In [18]:
# Remove missing values
raw_data.dropna(inplace= True)
raw_data.reset_index(drop= True, inplace= True)
# YOUR CODE HERE to check again for missing values
raw_data.isna().sum() / len(raw_data)

,0
Survey Year,0.0
Timestamp,0.0
SalaryUSD,0.0
Country,0.0
PostalCode,0.0
PrimaryDatabase,0.0
YearsWithThisDatabase,0.0
OtherDatabases,0.0
EmploymentStatus,0.0
JobTitle,0.0


In [19]:
# Data type of SalaryUSD column
raw_data['SalaryUSD'].dtype

dtype('O')

In [20]:
# Unique values of SalaryUSD column
raw_data['SalaryUSD'].unique()

array([200000, 95000, 47000, 41000, 51652, 63000, 45000, 123000, 80000,
       102000, 107690, 84000, 120000, 140000, 138000, 40000, 72800, 99000,
       50000, 55000, 175000, 113400, 97500, 17621, 105000, 85000, 104000,
       98000, 70760, 67770, 82000, 98517, 107000, 78000, 72000, 125000,
       67000, 129000, 66000, 121000, 10000, 15000, 130000, 87000, 70000,
       35900, 61606, 96000, 27000, 35000, 50400, 160000, 115000, 7968,
       138500, 63500, 88000, 111000, 134000, 95500, 100000, 122000, 26000,
       14000, 112500, 12000, 12900, 110000, 53000, 113000, 90000, 93000,
       94000, 65000, 92000, 120792, 145000, 93800, 93500, 270000, 108000,
       117000, 118500, 165000, 73000, 109000, 62000, 70400, 153000,
       116000, 149000, 75000, 76000, 142500, 98600, 62666, 146000, 148000,
       115544, 77000, 97000, 69725, 91520, 20000, 56000, 89500, 112350,
       55670, 137000, 81000, 150000, 53343, 74424, 135000, 67418, 127500,
       87491, 40100, 56140, 102500, 83000, 145493, 1

In [21]:
# Processing Salary column

def process_salary(salary):
    sep = '.'
    salary = str(salary)
    # Replace characters and take the cents out of our data
    salary = salary.replace(" ","").replace("$","").replace(",","").split(sep)[0]

    return float(salary)

# Replace spaces(“ “) in columns name with underscore (“_”)
raw_data.columns = raw_data.columns.str.replace(" ","_")

# Apply process_salary function
# YOUR CODE HERE to update 'SalaryUSD' column of 'raw_data' by applying 'process_salary' to its every element
raw_data['SalaryUSD'] = raw_data['SalaryUSD'].apply(process_salary)
raw_data['SalaryUSD'].head()

,SalaryUSD
0,200000.0
1,95000.0
2,47000.0
3,41000.0
4,51652.0


In order to analyze only 2019 United States data, we will filter the DataFrame by year and country.

In [22]:
# Filter dataframe by year
df = raw_data[raw_data.Survey_Year == 2019]
# YOUR CODE HERE to show few rows of 'df'
df.head()

,Survey_Year,Timestamp,SalaryUSD,Country,PostalCode,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,...,HoursWorkedPerWeek,TelecommuteDaysPerWeek,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed,Counter
1935,2019,2018-12-06 13:58:01.557,128500.0,United States,442,Microsoft SQL Server,15,"Microsoft SQL Server, Oracle",Full time employee,Architect,...,40,"None, or less than 1 day per week",300K-1M (large city),Private business,No,"Stay with the same employer, same role",Male,"DBA (Development Focus - tunes queries, indexe...","Manual tasks, Meetings & management, Projects",1
1936,2019,2018-12-11 06:24:30.227,110000.0,United States,43016,Microsoft SQL Server,18,Azure SQL DB,Full time employee,"DBA (Development Focus - tunes queries, indexe...",...,44,"None, or less than 1 day per week",1M+ (metropolis),Private business,No,"Stay with the same employer, same role",Male,DBA (Production Focus - build & troubleshoot s...,"Build scripts & automation tools, Manual tasks...",1
1937,2019,2018-12-11 06:33:56.358,124000.0,United States,92105,Microsoft SQL Server,18,Oracle,Full time employee,DBA (General - splits time evenly between writ...,...,40,4,1M+ (metropolis),Private business,"Yes, actively looking for something else","Stay with the same role, but change employers",Male,"DBA (Development Focus - tunes queries, indexe...","Manual tasks, Projects, R&D",1
1938,2019,2018-12-11 06:38:03.026,115000.0,United States,494,Microsoft SQL Server,14,"Microsoft SQL Server, Elasticsearch, Azure SQL DB",Full time employee,"DBA (Development Focus - tunes queries, indexe...",...,45,4,100K-299K (city),Private business,"Yes, actively looking for something else","Stay with the same role, but change employers",Male,DBA (Production Focus - build & troubleshoot s...,"Meetings & management, On-call as part of a ro...",1
1939,2019,2018-12-11 06:46:04.981,115000.0,United States,43016,Microsoft SQL Server,15,Azure SQL DB,Full time employee,"DBA (Development Focus - tunes queries, indexe...",...,55,"None, or less than 1 day per week",300K-1M (large city),Private business,"Yes, but only passively (just curious)","Stay with the same employer, same role",Male,"Architect, DBA (Production Focus - build & tro...","On-call as part of a rotation, Projects, R&D, ...",1


In [23]:
# Filter dataframe by country
US_2019 = df.loc[df.Country == 'United States', :]
US_2019.head(3)

,Survey_Year,Timestamp,SalaryUSD,Country,PostalCode,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,...,HoursWorkedPerWeek,TelecommuteDaysPerWeek,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed,Counter
1935,2019,2018-12-06 13:58:01.557,128500.0,United States,442,Microsoft SQL Server,15,"Microsoft SQL Server, Oracle",Full time employee,Architect,...,40,"None, or less than 1 day per week",300K-1M (large city),Private business,No,"Stay with the same employer, same role",Male,"DBA (Development Focus - tunes queries, indexe...","Manual tasks, Meetings & management, Projects",1
1936,2019,2018-12-11 06:24:30.227,110000.0,United States,43016,Microsoft SQL Server,18,Azure SQL DB,Full time employee,"DBA (Development Focus - tunes queries, indexe...",...,44,"None, or less than 1 day per week",1M+ (metropolis),Private business,No,"Stay with the same employer, same role",Male,DBA (Production Focus - build & troubleshoot s...,"Build scripts & automation tools, Manual tasks...",1
1937,2019,2018-12-11 06:33:56.358,124000.0,United States,92105,Microsoft SQL Server,18,Oracle,Full time employee,DBA (General - splits time evenly between writ...,...,40,4,1M+ (metropolis),Private business,"Yes, actively looking for something else","Stay with the same role, but change employers",Male,"DBA (Development Focus - tunes queries, indexe...","Manual tasks, Projects, R&D",1


In [37]:
# Show number of employees by their genders in US_2019 dataset
# YOUR CODE HERE
employees_by_genders = US_2019['Gender'].value_counts()
employees_by_genders

,count
Gender,
Male,170
Female,16


In [28]:
# Filter salary by gender
Male_salary = US_2019[US_2019.Gender == 'Male']['SalaryUSD']
# YOUR CODE HERE to create 'Female_salary'
Female_salary = US_2019[US_2019.Gender == 'Female']['SalaryUSD']

print("Mean salary for male employees: ", Male_salary.mean())
print("Mean salary for female employees: ", Female_salary.mean())

Mean salary for male employees:  107485.23529411765
Mean salary for female employees:  95481.25


In [38]:
# Perform t-test on data
ttest = stats.ttest_ind(a = Female_salary, b = Male_salary, equal_var= False)
ttest

TtestResult(statistic=-1.2238287711542442, pvalue=0.23553795212550016, df=19.583191021914505)

# Since p-value > 0.05, we fail to reject null hypothesis $H_0$ : salaries for male and female are equal.

Discuss the p-value achieved in the above result and put your inference.

### References for further reading:

[Hypothesis testing in machine learning](https://towardsdatascience.com/hypothesis-testing-in-machine-learning-using-python-a0dc89e169ce)

[How to use z-table](https://www.statology.org/how-to-use-z-table/)


### Please answer the questions below to complete the experiment:




In [30]:
# @title Select the correct option below, based on the following statement: The p-value is defined as the smallest value of alpha for which the null hypothesis can be rejected. { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "If the p-value is less than or equal to alpha, we reject the null hypothesis" #@param ["","If the p-value is less than or equal to alpha, we reject the null hypothesis","If the p-value is greater than alpha, we fail to reject the null hypothesis","Both of the above", "None of the above"]

In [40]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good, But Not Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [31]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "More code snippets need to be given for practice." #@param {type:"string"}


In [42]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [32]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [33]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [43]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 725
Date of submission:  30 Nov 2024
Time of submission:  16:40:37
View your submissions: https://learn-iisc.talentsprint.com/notebook_submissions
